In [113]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LambdaCallback

In [88]:
# Wczytywanie danych
data = pd.read_csv("data/train.csv")
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Podział danych
y = data['label']
X = data.drop(columns=['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
# Normalizacja danych
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
# model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


TypeError: Invalid keyword argument(s) in `compile()`: ({'learning_rate'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

In [ ]:
epoch_results = []
all_predictions = []

def evaluate_and_save(epoch, logs):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    train_loss = logs['loss']
    train_accuracy = logs['accuracy']
    val_loss = logs['val_loss']
    val_accuracy = logs['val_accuracy']
    
    epoch_results.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_accuracy': train_accuracy,
        'val_loss': val_loss,
        'val_accuracy': val_accuracy,
        'test_loss': loss,
        'test_accuracy': accuracy
    })

    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)

    epoch_predictions = {
        'epoch': epoch + 1,
        'true_labels': y_test,
        'predicted_labels': predicted_classes,
        'predictions': predictions.tolist()  # Predykcje jako lista
    }

    all_predictions.append(epoch_predictions)

    #model.save(f'model/epoch_{epoch + 1}_score_{accuracy:.2f}.h5')

eval_and_save_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: evaluate_and_save(epoch, logs))

history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    callbacks=[eval_and_save_callback]
)


Epoch 1/10
1005/1005 [==============================] - 8s 8ms/step - loss: 0.0059 - accuracy: 0.9988 - val_loss: 0.1893 - val_accuracy: 0.9815
Epoch 2/10
1005/1005 [==============================] - 8s 8ms/step - loss: 0.0098 - accuracy: 0.9976 - val_loss: 0.1815 - val_accuracy: 0.9821
Epoch 3/10
1005/1005 [==============================] - 8s 8ms/step - loss: 0.0072 - accuracy: 0.9982 - val_loss: 0.1801 - val_accuracy: 0.9852
Epoch 4/10
1005/1005 [==============================] - 10s 10ms/step - loss: 0.0065 - accuracy: 0.9984 - val_loss: 0.1722 - val_accuracy: 0.9857
Epoch 5/10
1005/1005 [==============================] - 9s 9ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 0.1786 - val_accuracy: 0.9860
Epoch 6/10
1005/1005 [==============================] - 7s 7ms/step - loss: 0.0094 - accuracy: 0.9983 - val_loss: 0.1826 - val_accuracy: 0.9849
Epoch 7/10
1005/1005 [==============================] - 7s 7ms/step - loss: 0.0035 - accuracy: 0.9991 - val_loss: 0.1753 - val_accurac

In [121]:
model.evaluate(X_test, y_test)

197/197 [==============================] - 1s 5ms/step - loss: 0.1840 - accuracy: 0.9851


[0.18399780988693237, 0.9850793480873108]

In [122]:
epoch_results_df = pd.DataFrame(epoch_results)
all_predictions_df = pd.DataFrame(all_predictions)

epoch_results_df
#all_predictions_df

,epoch,train_loss,train_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
0,1,0.005862,0.998817,0.189349,0.981513,0.182565,0.983333
1,2,0.009819,0.997603,0.181494,0.982073,0.187481,0.983810
2,3,0.007183,0.998195,0.180147,0.985154,0.206519,0.984921
3,4,0.006474,0.998382,0.172157,0.985714,0.201952,0.984286
4,5,0.004727,0.998786,0.178609,0.985994,0.188281,0.986825
5,6,0.009400,0.998288,0.182557,0.984874,0.186094,0.984762
6,7,0.003535,0.999129,0.175285,0.985994,0.198092,0.983810
7,8,0.003616,0.999160,0.153432,0.986274,0.201441,0.985238
8,9,0.004770,0.998631,0.161371,0.986555,0.218732,0.984603
9,10,0.003919,0.999035,0.160040,0.987115,0.183998,0.985079
